In [0]:
# Databricks notebook source
from pyspark.sql import SparkSession, functions as f

file_path_hospital1 = "abfss://bronze@emrgen2.dfs.core.windows.net/Hosptial1/providers"
df_hospital1 = spark.read.parquet(file_path_hospital1)
file_path_hospital10 = "abfss://bronze@emrgen2.dfs.core.windows.net/hospital10/providers"
df_hospital10 = spark.read.parquet(file_path_hospital10)

df_merged = df_hospital10.union(df_hospital1)
display(df_merged)

df_merged.createOrReplaceTempView("providers")

In [0]:

%sql
CREATE TABLE IF NOT EXISTS silver.providers (
ProviderID string,
FirstName string,
LastName string,
Specialization string,
DeptID string,
NPI long,
datasource string,
is_quarantined boolean
)
USING DELTA;

In [0]:

%sql
truncate table silver.providers

In [0]:
%sql 
insert into silver.providers
select 
distinct
ProviderID,
FirstName,
LastName,
Specialization,
DeptID,
 NPI,
datasource,
    CASE 
        WHEN ProviderID IS NULL OR DeptID IS NULL THEN TRUE
        ELSE FALSE
    END AS is_quarantined
from providers


In [0]:
%sql
select * from silver.providers